# Exercise 04. A/B-testing

In [123]:
import pandas as pd
import sqlite3

## Create a connection to the database using the `sqlite3` library

In [124]:
con = sqlite3.connect("../data/checking-logs.sqlite")

## Using only one query for each group, create two dataframes: `test_results` and `control_results`, with the columns "time" and "avg_diff", and only two rows

In [125]:
query = """
WITH test_results AS
(
    SELECT uid, time, delta
    FROM
    (
        SELECT uid,
            (CAST(strftime('%s', first_commit_ts) AS INT) - deadlines) / 3600
            AS delta,
            CASE
                WHEN t.first_commit_ts < t.first_view_ts
                THEN 'before'
                ELSE 'after'
            END AS time
        FROM test t
        JOIN deadlines dl
        ON t.labname = dl.labs
        WHERE NOT t.labname = 'project1'
    )
)
SELECT time, AVG(delta) AS avg_diff
FROM test_results
WHERE uid IN 
(
    SELECT uid
    FROM
    (
        SELECT uid, time
        FROM test_results
    )
    GROUP BY uid
    HAVING COUNT(DISTINCT time) = 2
)
GROUP BY time
"""
test_results = pd.io.sql.read_sql(query, con)
test_results

,time,avg_diff
0,after,-104.6000
1,before,-60.5625


In [126]:
query = """
WITH control_results AS
(
    SELECT uid, time, delta
    FROM
    (
        SELECT uid,
            (CAST(strftime('%s', first_commit_ts) AS INT) - deadlines) / 3600
            AS delta,
            CASE
                WHEN c.first_commit_ts < c.first_view_ts
                THEN 'before'
                ELSE 'after'
            END AS time
        FROM control c
        JOIN deadlines dl
        ON c.labname = dl.labs
        WHERE NOT c.labname = 'project1'
    )
)
SELECT time, AVG(delta) AS avg_diff
FROM control_results
WHERE uid IN 
(
    SELECT uid
    FROM
    (
        SELECT uid, time
        FROM control_results
    )
    GROUP BY uid
    HAVING COUNT(DISTINCT time) = 2
)
GROUP BY time
"""
control_results = pd.io.sql.read_sql(query, con)
control_results

,time,avg_diff
0,after,-117.636364
1,before,-99.464286


## Close the connection

In [127]:
con.close()

## Have the answer: Did the hypothesis turn out to be true, and does the page affect students' behavior?

The answer: yes, because the test group data has a big difference when the control group values are closer.